In [244]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
from bs4 import BeautifulSoup
import sys
sys.path.append('../../assets/python/')
import tfb

pd.options.mode.chained_assignment = None  # default='warn'

METADATA = {'Contributor': 'T. Dunn'}
SAVEFIGS = True

df0 = pd.read_csv('../../assets/data/monsters.csv')
print('Includes {} monsters in database'.format(len(df0.index.to_list())))

Includes 1904 monsters in database


In [245]:
# functions
import json
import pandas as pd
import numpy as np
from scipy import optimize

def average_save_bonus(row, **kwargs):
    saves = kwargs.get('saves', ['Str Save','Dex Save','Con Save','Int Save','Wis Save','Cha Save'])
    ave = np.mean([row[s] for s in saves])
    return ave

def average_save_mod(row, **kwargs):
    saves = kwargs.get('saves', ['Str Mod','Dex Mod','Con Mod','Int Mod','Wis Mod','Cha Mod'])
    ave = np.mean([row[s] for s in saves])
    return ave

def average_effective_save_bonus(row, **kwargs):
    ave = average_save_bonus(row, **kwargs)
    ave += 4 if 'magic resistance' in row['AC adjustments'] else 0
    return ave

def average_effective_save_mod(row, **kwargs):
    ave = average_save_mod(row, **kwargs)
    ave += 4 if 'magic resistance' in row['AC adjustments'] else 0
    return ave


def count_save_proficiencies(row):
    return row['Str Prof'] + row['Dex Prof'] + row['Con Prof'] + row['Int Prof'] + row['Wis Prof'] + row['Cha Prof']

def adjust_armor_class(row):
    n_saves = count_save_proficiencies(row)
    if n_saves >= 5:
        return 4
    elif n_saves >= 3:
        return 2
    else:
        return 0
    
def adjusted_armor_class(row):
    ac = row['AC']
    ac += 2 if 'magic resistance' in row['AC adjustments'] else 0
    match count_save_proficiencies(row):
        case 3:
            ac += 2
        case 4:
            ac += 2
        case 5:
            ac += 4
        case 6:
            ac += 4
        case _:
            ac += 0
    return ac

def unadjusted_armor_class(row):
    ac = row['adj AC']
    ac -= 2 if 'magic resistance' in row['AC adjustments'] else 0
    match count_save_proficiencies(row):
        case 3:
            ac -= 2
        case 4:
            ac -= 2
        case 5:
            ac -= 4
        case 6:
            ac -= 4
        case _:
            ac += 0
    return ac


def load_pc_data(file, columns):
    """Loads PC data from file and puts it into a pandas dataframe
    """
    with open(file, 'r') as fin:
        pc_data = json.load(fin)

    pc_dict = {}
    for c in columns:
        d = []
        for pc in pc_data:
            d += [x[c] for x in pc_data[pc]]
        pc_dict[c] = d

    return pd.DataFrame(pc_dict)


def plot_data_and_fit(fig, x, y, **kwargs):
    # plot data
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers', 
        **kwargs,
    ))

    # plot fit
    kwargs['hoverinfo'] = 'skip'
    kwargs['showlegend'] = False
    if kwargs.get('line', None):
        kwargs['line']['dash'] = 'solid'

    coefs = np.polyfit(x, y, 1)
    poly = np.poly1d(coefs)
    fig.add_trace(go.Scatter(
        x=x, y=poly(x),
        mode='lines', 
        **kwargs,
    ))
    return fig

def piecewise_linear(x, x0, y0, k1, k2):
    #return np.piecewise(x, [x < x0], [lambda x:k1*x + y0-k1*x0, lambda x:k2*x + y0-k2*x0])
    return np.piecewise(x, [x < 20], [lambda x:k1*x + y0-k1*20, lambda x:k2*x + y0-k2*20])

def plot_data_and_piecewise_fit(fig, x, y, **kwargs):
    # plot data
    fig.add_trace(go.Scatter(
        x=x, y=y,
        mode='markers', 
        **kwargs,
    ))

    # plot fit
    kwargs['hoverinfo'] = 'skip'
    kwargs['showlegend'] = False
    kwargs['line_dash'] = 'solid'
    p, e = optimize.curve_fit(piecewise_linear, x, y)
    fig.add_trace(go.Scatter(
        x=x, 
        y=piecewise_linear(x, *p),
        mode='lines', 
        **kwargs,
    ))
    return fig, piecewise_linear(np.linspace(1,30,30), *p)

def attack_hit_crit_prob(AC, AB):
    return max(0.05, min(0.95, 0.05*(21 + AB - AC)))

def save_fail_prob(SB, DC):
    return max(0.0, min(1.0, 0.05*(DC - SB - 1)))

COLOR_LIST = [
    '#1f77b4',  # muted blue
    '#ff7f0e',  # safety orange
    '#2ca02c',  # cooked asparagus green
    '#d62728',  # brick red
    '#9467bd',  # muted purple
    '#8c564b',  # chestnut brown
    '#e377c2',  # raspberry yogurt pink
    '#7f7f7f',  # middle gray
    '#bcbd22',  # curry yellow-green
    '#17becf'   # blue-teal
]

In [246]:
# load PC data
PC_DATA = {
    'level':      np.array([ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20]),
    'attack mod': np.array([ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]),
    'prof bonus': np.array([ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6]),
    "armor class": {
        "Barbarian": np.array([15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 17, 17, 17, 18, 20]),
        "Bard":      np.array([14, 14, 14, 14, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]),
        "Cleric":    np.array([18, 18, 18, 18, 18, 18, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]),
        "Druid":     np.array([15, 15, 15, 16, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]),
        "Fighter":   np.array([17, 17, 17, 17, 17, 17, 17, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19, 19]),
        "Monk":      np.array([16, 16, 16, 17, 17, 17, 17, 18, 18, 18, 18, 19, 19, 19, 19, 20, 20, 24, 24, 24]),
        "Paladin":   np.array([16, 16, 16, 16, 16, 16, 16, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18, 18]),
        "Ranger":    np.array([14, 14, 14, 15, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]),
        "Rogue":     np.array([14, 14, 14, 15, 16, 16, 16, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17, 17]),
        "Sorcerer":  np.array([12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]),
        "Warlock":   np.array([13, 13, 13, 13, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15, 15]),
        "Wizard":    np.array([12, 12, 12, 12, 15, 15, 15, 15, 15, 15, 16, 16, 16, 16, 16, 16, 16, 16, 16, 16]),
    },
    'ability mod': {
        'ability 1': np.array([ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5]),
        'ability 2': np.array([ 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
        'ability 3': np.array([ 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
        'ability 4': np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]),
        'ability 5': np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]),
        'ability 6': np.array([-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1,-1]),
    },
    'proficiencies': {
        "Barbarian": np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        "Bard":      np.array([2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
        "Cleric":    np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        "Druid":     np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        "Fighter":   np.array([2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
        "Monk":      np.array([2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 7, 7, 7, 7, 7, 7, 7]),
        "Paladin":   np.array([2, 2, 2, 2, 2, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8]),
        "Ranger":    np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        "Rogue":     np.array([2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 4, 4, 4, 4, 4, 4]),
        "Sorcerer":  np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
        "Warlock":   np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]),
        "Wizard":    np.array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3]),
    }
}
PC_DATA['attack bonus'] = PC_DATA['attack mod'] + PC_DATA['prof bonus']
PC_DATA['save dc'] = 8 + PC_DATA['attack bonus']

ac_mean = np.zeros(20)
for c in PC_DATA['armor class']:
    ac_mean += PC_DATA['armor class'][c]
ac_mean /= len(PC_DATA['armor class'])
PC_DATA['armor class'] = ac_mean

ave_mod = np.zeros(20)
for ab in PC_DATA['ability mod']:
    ave_mod += PC_DATA['ability mod'][ab]
ave_mod = ave_mod/6
ave_prof = np.zeros(20)
for c in PC_DATA['proficiencies']:
    ave_prof += PC_DATA['proficiencies'][c]
ave_prof /= len(PC_DATA['proficiencies'])

PC_DATA['save mod'] = list(ave_mod)
PC_DATA['save bonus'] = ave_mod + PC_DATA['prof bonus']*ave_prof/6
PC_DATA['save AC'] = PC_DATA['save bonus'] + 13
PC_DATA['eAC'] = 0.5*(PC_DATA['armor class'] + PC_DATA['save AC'])

In [247]:
# plots monster DPR vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='damage per round',
            range=[0,320],
            tick0=0, dtick=50,
            minor=dict(tick0=0, dtick=10),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# get monster data
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[['CR','DPR','adj DPR']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]

# plot adjusted damage per round
name = 'adj DPR'
fig, y = plot_data_and_piecewise_fit(
    fig, dfM['CR'].to_numpy(), 
    dfM[name].to_numpy(), 
    name='monsters (adj)', legendgroup='monsters (adj)', 
    line_color=COLOR_LIST[0],
    marker_color=COLOR_LIST[0],
    hovertemplate=
        '<b>monsters (adj)</b><br>'+
        'CR %{x}<br>'+
        'DPR %{y:,.1f}' + 
        '<extra></extra>'
)

# plot unadjusted damage per round
name = 'DPR'
fig, y = plot_data_and_piecewise_fit(
    fig, dfM['CR'].to_numpy(), 
    dfM[name].to_numpy(), 
    name='monsters (unadj)', legendgroup='monsters (unadj)', 
    line_color=COLOR_LIST[1],
    marker_color=COLOR_LIST[1],
    hovertemplate=
        '<b>monsters (unadj)</b><br>'+
        'CR %{x}<br>'+
        'DPR %{y:,.1f}' + 
        '<extra></extra>'
)

# plot reference line
dfD0 = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','XP','HP Mean','AC','DPR Mean','AB'
dfD0 = dfD0[dfD0['CR'].between(1,30)]
fig.add_trace(go.Scatter(
    x=dfD0['CR'], y=dfD0['DPR Mean'],
    mode='markers+lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</b><br>'+
        'CR %{x}<br>'+
        'DPR %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-dpr-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-dpr-vs-cr-small')

/Users/tom/Development/python-venv/thefinishedbook/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning:

Covariance of the parameters could not be estimated



In [231]:
# plots average attack bonus
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='attack bonus',
            range=[2,20],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


# plot monster adjusted average
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[['CR','AC','AB','adj AB','Save DC']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]
plot_data_and_fit(
    fig, 
    dfM['CR'], dfM['adj AB'], 
    name='monsters (adj)',
    legendgroup='monsters (adj)',
    line=dict(color=COLOR_LIST[0]),
    hovertemplate=
        '<b>monsters (adj)</b><br>'+
        'CR %{x}<br>'+
        'AB %{y:,.1f}' + 
        '<extra></extra>'
)

# plot monster unadjusted average
plot_data_and_fit(
    fig, 
    dfM['CR'], dfM['AB'], 
    name='monsters (unadj)',
    legendgroup='monsters (unadj)',
    line=dict(color=COLOR_LIST[1]),
    hovertemplate=
        '<b>monsters (unadj)</b><br>'+
        'CR %{x}<br>'+
        'AB %{y:,.1f}' + 
        '<extra></extra>'
)

# plot monster DMG targets
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 30)]
fig.add_trace(go.Scatter(
    x=np.array(dfD['CR']), 
    y=np.array(dfD['AB']),
    mode='markers+lines', 
    name='DMG',
    legendgroup='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</b><br>'+
        'CR %{x}<br>'+
        'AB %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-ab-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-ab-vs-cr-small')

In [232]:
# plots average save difficulty class
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save difficulty class',
            range=[10,28],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


# plot monster average
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[['CR','AC','AB','adj AB','Save DC']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]


plot_data_and_fit(
    fig, 
    dfM['CR'], dfM['Save DC'], 
    name='monsters (adj)',
    legendgroup='monsters (adj)',
    line=dict(color=COLOR_LIST[0]),
    hovertemplate=
        '<b>monsters (adj)</b><br>'
        'CR %{x}<br>'+
        'Save DC %{y:,.1f}' + 
        '<extra></extra>'
)

plot_data_and_fit(
    fig, 
    dfM['CR'], dfM['Save DC'], 
    name='monsters (unadj)',
    legendgroup='monsters (unadj)',
    line=dict(color=COLOR_LIST[1]),
    hovertemplate=
        '<b>monsters (unadj)</b><br>'
        'CR %{x}<br>'+
        'Save DC %{y:,.1f}' + 
        '<extra></extra>'
)


# plot monster DMG targets
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 30)]
fig.add_trace(go.Scatter(
    x=np.array(dfD['CR']), 
    y=np.array(dfD['DC']),
    mode='markers+lines', 
    name='DMG',
    legendgroup='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</b><br>'
        'CR %{x}<br>'+
        'Save DC %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-dc-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-dc-vs-cr-small')

In [233]:
# plots average save DC vs AB for monsters and the DMG
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='attack bonus',
            range=[2,20],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save difficulty class',
            range=[10,28],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


# plot monster average
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[['CR','AC','AB','adj AB','Save DC']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]


plot_data_and_fit(
    fig, 
    dfM['adj AB'], dfM['Save DC'], 
    name='monsters (adj)',
    legendgroup='monsters (adj)',
    line=dict(color=COLOR_LIST[0]),
    hovertemplate=
        '<b>monsters (adj)</b><br>'
        'AB %{x:,.1f}<br>'+
        'Save DC %{y:,.1f}' + 
        '<extra></extra>'
)


# plot monster DMG targets
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 30)]
fig.add_trace(go.Scatter(
    x=np.array(dfD['AB']), 
    y=np.array(dfD['DC']),
    mode='markers+lines', 
    name='DMG',
    legendgroup='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</b><br>'
        'AB %{x:,.1f}<br>'+
        'Save DC %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-dc-vs-ab-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-dc-vs-ab-small')

In [234]:
# plots average attack bonus for both monster and player characters
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='attack bonus',
            range=[2,20],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster average
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[['CR','adj AB']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]
plot_data_and_fit(
    fig, 
    #dfM['CR'], dfM['AB'] + 8, 
    dfM['CR'], dfM['adj AB'], 
    name='monsters (adj)',
    legendgroup='monsters (adj)',
    line=dict(color=COLOR_LIST[0]),
    hovertemplate=
        '<b>monsters (adj)</b><br>'+
        'CR %{x}<br>'+
        'AB %{y:,.0f}' + 
        '<extra></extra>'
)

# plot player character save DCs
pc = {
    'level':      [ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20],
    'attack mod': [ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'prof bonus': [ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
}
pc['attack bonus'] = [pc['attack mod'][i] + pc['prof bonus'][i] for i in range(20)]
pc['save dc'] = [pc['attack bonus'][i] + 8 for i in range(20)]
"""
plot_data_and_fit(
    fig, 
    np.array(pc['level']), np.array(pc['attack bonus']), 
    name='player characters',
    legendgroup='player characters',
    line=dict(color=COLOR_LIST[5]),
    hovertemplate=
        '<b>player characters</p><br>'+
        'Level %{x}<br>'+
        'AB %{y:,.0f}' + 
        '<extra></extra>'
)
"""
fig.add_trace(go.Scatter(
    x=np.array(pc['level']), 
    y=np.array(pc['attack bonus']),
    mode='markers+lines', 
    name='player characters',
    line_color=COLOR_LIST[5],
    marker_color=COLOR_LIST[5],
    hovertemplate=
        '<b>player characters</p><br>'+
        'Level %{x}<br>'+
        'AB %{y:,.0f}' + 
        '<extra></extra>'
))

# plot monster save DCs
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 30)]
fig.add_trace(go.Scatter(
    x=np.array(dfD['CR']), 
    y=np.array(dfD['AB']),
    mode='markers+lines', 
    name='DMG',
    legendgroup='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</p><br>'+
        'CR %{x}<br>'+
        'AB %{y:,.0f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-ab-vs-level-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-ab-vs-level-small')

In [248]:
# plots monster effective DPR vs CR
import sys
sys.path.append('../../assets/python/')
import estats5e as e5

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='effective damage per round',
            range=[0,500],
            tick0=0, dtick=100,
            minor=dict(tick0=0, dtick=20),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# get monster data
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM['eDPR'] = dfM.apply(lambda row: e5.effDPR(row['adj DPR'], row['adj AB'], 'linear'), axis=1)
dfM = dfM[['CR','eDPR']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]

# plot adjusted damage per round
fig, y = plot_data_and_piecewise_fit(
    fig, dfM['CR'].to_numpy(), 
    dfM['eDPR'].to_numpy(), 
    name='monsters', legendgroup='monsters', 
    line_color=COLOR_LIST[0],
    marker_color=COLOR_LIST[0],
    hovertemplate=
        '<b>monsters</b><br>'
        'CR %{x}<br>'+
        'eff DPR %{y:,.1f}' + 
        '<extra></extra>'
)

# plot reference line
dfD0 = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','XP','HP Mean','AC','DPR Mean','AB'
dfD0['eDPR'] = dfD0.apply(lambda row: e5.effDPR(row['DPR Mean'], row['AB'], 'linear'), axis=1)
dfD0 = dfD0[dfD0['CR'].between(1,30)]
fig.add_trace(go.Scatter(
    x=dfD0['CR'], y=dfD0['eDPR'],
    mode='markers+lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    #hoverinfo='skip', 
    hovertemplate=
        '<b>DMG</b><br>'
        'CR %{x}<br>'+
        'eff DPR %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-eff-dpr-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-eff-dpr-vs-cr-small')

/Users/tom/Development/python-venv/thefinishedbook/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning:

Covariance of the parameters could not be estimated



In [250]:
# plots monster HP vs CR

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='hit points',
            range=[0,900],
            tick0=0, dtick=100,
            minor=dict(tick0=0, dtick=50),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[['CR','HP','adj HP']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 31)]

# plot adjusted hit points
name = 'adj HP'
fig, y = plot_data_and_piecewise_fit(
    fig, dfM['CR'].to_numpy(), 
    dfM[name].to_numpy(), 
    name='monsters (adj)', legendgroup='monsters (adj)', 
    line_color=COLOR_LIST[0],
    marker_color=COLOR_LIST[0],
    hovertemplate=
        '<b>monsters (adjusted)</b><br>'+
        'CR %{x}<br>'+
        'HP %{y:,.1f}' + 
        '<extra></extra>'
)

# plot hit points
name = 'HP'
fig, y = plot_data_and_piecewise_fit(
    fig, dfM['CR'].to_numpy(), 
    dfM[name].to_numpy(), 
    name='monsters (unadj)', legendgroup='monsters (unadj)', 
    line_color=COLOR_LIST[1],
    marker_color=COLOR_LIST[1],
    hovertemplate=
        '<b>monsters (unadjusted)</b><br>'+
        'CR %{x}<br>'+
        'HP %{y:,.1f}' + 
        '<extra></extra>'
)

# plot reference line
dfD0 = pd.read_csv('../../assets/data/dmg-targets.csv')
dfD0 = dfD0[dfD0['CR'].between(1,30)]
fig.add_trace(go.Scatter(
    x=dfD0['CR'], y=dfD0['HP Mean'],
    mode='markers+lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</b><br>'+
        'CR %{x}<br>'+
        'HP %{y:,.1f}' + 
        '<extra></extra>' 
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-hp-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-hp-vs-cr-small')

/Users/tom/Development/python-venv/thefinishedbook/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning:

Covariance of the parameters could not be estimated



In [237]:
# plots average armor class 
# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='armor class',
            range=[10,30],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster armor class
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM['AC adj'] = dfM.apply(lambda row: adjusted_armor_class(row), axis=1)
dfM = dfM[['CR','AC','adj AC','AC adj']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]
plot_data_and_fit(
    fig, 
    dfM['CR'], dfM['adj AC'], 
    name='monsters (adj)',
    legendgroup='monsters (adj)',
    line=dict(color=COLOR_LIST[0]),
    hovertemplate=
        '<b>monsters</b><br>'
        'CR %{x}<br>'+
        'AC %{y:,.1f}' + 
        '<extra></extra>'
)

plot_data_and_fit(
    fig, 
    dfM['CR'], dfM['AC'], 
    name='monsters (unadj)',
    legendgroup='monsters (unadj)',
    line=dict(color=COLOR_LIST[1]),
    hovertemplate=
        '<b>monsters</b><br>'
        'CR %{x}<br>'+
        'AC %{y:,.1f}' + 
        '<extra></extra>'
)


# plot monster armor class targets
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 30)]
fig.add_trace(go.Scatter(
    x=np.array(dfD['CR']), 
    y=np.array(dfD['AC']),
    mode='markers+lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</b><br>'
        'CR %{x}<br>'+
        'AC %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-ac-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-ac-vs-cr-small')

In [238]:
# plots average save bonus

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save bonus',
            range=[-4,16],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot published monsters
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[dfM['CR'].between(1,30)]
dfM['SB']     = dfM.apply(lambda row: average_save_bonus(row), axis=1)
dfM['adj SB'] = dfM['SB'] + dfM.apply(lambda row: 4 if 'magic resistance' in row['AC adjustments'] else 0, axis=1)
dfM = dfM[['CR','SB','adj SB']].groupby('CR').mean().reset_index()
dfM.sort_values(by='CR', inplace=True)

plot_data_and_fit(
    fig, 
    dfM['CR'], dfM['adj SB'], 
    name='monsters (adj)',
    legendgroup='monsters (adj)',
    line=dict(color=COLOR_LIST[0]),
    hovertemplate=
        '<b>monsters</b><br>'+
        'CR %{x}<br>'+
        'SB %{y:,.0f}' + 
        '<extra></extra>'
)

plot_data_and_fit(
    fig, 
    dfM['CR'], dfM['SB'], 
    name='monsters (unadj)',
    legendgroup='monsters (unadj)',
    line=dict(color=COLOR_LIST[1]),
    hovertemplate=
        '<b>monsters</b><br>'+
        'CR %{x}<br>'+
        'SB %{y:,.0f}' + 
        '<extra></extra>'
)

# plot DMG targets
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 30)]
fig.add_trace(go.Scatter(
    x=np.array(dfD['CR']), 
    y=np.array(dfD['AC']) - 14,
    #y=np.array(dfD['DC']) - 14,
    mode='markers+lines', 
    name='DMG (AC - 14)',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</b><br>'+
        'CR %{x}<br>'+
        '(AC-14) %{y:,.0f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-sb-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-sb-vs-cr-small')

In [239]:
# plots average save bonus

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='armor class',
            range=[10,30],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)


# plot published monsters
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[dfM['CR'].between(1,30)]
dfM['unadj AC'] = dfM.apply(lambda row: unadjusted_armor_class(row), axis=1)
dfM['Ave Save'] = dfM.apply(lambda row: average_save_bonus(row), axis=1)
dfM['Ave Save'] = dfM['Ave Save'] + dfM.apply(lambda row: 4 if 'magic resistance' in row['AC adjustments'] else 0, axis=1)
dfM = dfM[['CR','Ave Save','AC','adj AC','unadj AC']].groupby('CR').mean().reset_index()
dfM.sort_values(by='CR', inplace=True)

plot_data_and_fit(
    fig, 
    dfM['CR'], dfM['adj AC'], 
    name='monsters (adj AC)',
    legendgroup='monsters (adj AC)',
    line=dict(color=COLOR_LIST[0]),
    hovertemplate=
        '<b>monsters (adj AC)</b><br>'
        'CR %{x}<br>'+
        'AC %{y:,.1f}' + 
        '<extra></extra>'
)

plot_data_and_fit(
    fig, 
    dfM['CR'], 0.5*(dfM['Ave Save'] + 14 + dfM['unadj AC']), 
    name='monsters (unadj AC + adj SB)/2',
    legendgroup='monsters (unadj AC + adj SB)/2',
    line=dict(color=COLOR_LIST[3], dash='dash'),
    hovertemplate=
        '<b>monsters (unadj AC + adj SB)/2</b><br>'+
        'CR %{x}<br>'+
        'AC %{y:,.1f}' + 
        '<extra></extra>'
)

# plot monster armor class targets
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 30)]
fig.add_trace(go.Scatter(
    x=np.array(dfD['CR']), 
    y=np.array(dfD['AC']),
    mode='markers+lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</b><br>'
        'CR %{x}<br>'+
        'AC %{y:,.1f}' + 
        '<extra></extra>'
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-adj-ac-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-adj-ac-vs-cr-small')

In [240]:
# plots probability of a PC hitting a monster with an attack
import json

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='probability',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# load PC data
pc = {
    'level':      [ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20],
    'attack mod': [ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'prof bonus': [ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
}
pc['attack bonus'] = [pc['attack mod'][i] + pc['prof bonus'][i] for i in range(20)]
pc['save dc'] = [pc['attack bonus'][i] + 8 for i in range(20)]


# plot chance to hit vs published monsters
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM['adj SB'] = dfM.apply(lambda row: average_effective_save_bonus(row), axis=1)
dfM = dfM[['CR','AC','adj SB']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]
level = np.array(pc['level'])
ab = np.array(pc['attack bonus'])
ac = dfM['AC'].to_numpy()
prob = np.array([attack_hit_crit_prob(a, b) for a, b in zip(ac, ab)])
fig.add_trace(go.Scatter(
    x=level, 
    y=prob,
    mode='markers+lines', 
    name='attacks',
    hovertemplate=
        '<b>Attacks</b><br>'+
        'Level %{x}<br>'+
        'Hit Probability %{y:,.1%}' + 
        '<extra></extra>'
))
print(prob.mean())

# plot chance to hit vs published monsters
level = np.array(pc['level'])
dc = np.array(pc['save dc'])
sb = dfM['adj SB'].to_numpy()
prob = np.array([save_fail_prob(a, b) for a, b in zip(sb, dc)])
fig.add_trace(go.Scatter(
    x=level, 
    y=prob,
    mode='markers+lines', 
    name='saving throws',
    hovertemplate=
        '<b>Saving Throws</b><br>'+
        'Level %{x}<br>'+
        'Fail Probability %{y:,.1%}' + 
        '<extra></extra>'
))
print(prob.mean())

# plot reference
fig.add_trace(go.Scatter(
    x=[1,20], 
    y=[0.65, 0.65],
    mode='lines', 
    name='monsters',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash'),
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-hit-prob-vs-level-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-hit-prob-vs-level-small')

0.6558122248594012
0.5364937124359251


In [241]:
# plots probability of a PC hitting a monster with a saving throw
import json

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save fail probability',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='bottom',
            x=0.00, y=0.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# load PC data
pc = {
    'level':      [ 1, 2, 3, 4, 5, 6, 7, 8, 9,10,11,12,13,14,15,16,17,18,19,20],
    'attack mod': [ 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5, 5],
    'prof bonus': [ 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6],
}
pc['attack bonus'] = [pc['attack mod'][i] + pc['prof bonus'][i] for i in range(20)]
pc['save dc'] = [pc['attack bonus'][i] + 8 for i in range(20)]


# plot chance to hit vs published monsters
saves = ['Str Save','Dex Save','Con Save','Int Save','Wis Save','Cha Save']
colors = ['#d62728','#9467bd','#8c564b','#1f77b4','#2ca02c','#ff7f0e']
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM['SB'] = dfM.apply(lambda row: average_save_bonus(row), axis=1)
dfM['adj SB'] = dfM.apply(lambda row: average_effective_save_bonus(row), axis=1)
dfM = dfM[['CR'] + saves].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 20)]
for save, color in zip(saves, colors):
    level = np.array(pc['level'])
    dc = np.array(pc['save dc'])
    sb = dfM[save].to_numpy()
    prob = np.array([save_fail_prob(a, b) for a, b in zip(sb, dc)])
    fig.add_trace(go.Scatter(
        x=level, 
        y=prob,
        mode='markers+lines', 
        name=save,
        line_color=color,
        marker_color=color,
        hovertemplate=
            f'<b>{save}</b><br>'+
            'Level %{x}<br>'+
            'Fail Probability %{y:,.1%}' + 
            '<extra></extra>'
    ))
    print(prob.mean())



# plot reference
fig.add_trace(go.Scatter(
    x=[1,20], 
    y=[0.65, 0.65],
    mode='lines', 
    name='monsters',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash'),
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name=f'./fig-save-fail-prob-vs-level-large')
    tfb.save_fig_html(fig, format='small', name=f'./fig-save-fail-prob-vs-level-small')

0.571468669470699
0.6228997107448316
0.49350055763258294
0.6708661761241038
0.5565587939404096
0.5926735240254851


In [249]:
# plots monster effective HP vs CR
import sys
sys.path.append('../../assets/python/')
import estats5e as e5

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=70, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='challenge rating',
            range=[0,31],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='effective hit points',
            range=[0,2400],
            tick0=0, dtick=200,
            minor=dict(tick0=0, dtick=100),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# get monster data
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM['unadj AC'] = dfM.apply(lambda row: unadjusted_armor_class(row), axis=1)
dfM['Ave Save'] = dfM.apply(lambda row: average_save_bonus(row), axis=1)
dfM['Ave Save'] = dfM['Ave Save'] + dfM.apply(lambda row: 4 if 'magic resistance' in row['AC adjustments'] else 0, axis=1)
dfM['eff AC'] = 0.5*(dfM['Ave Save']+14 + dfM['unadj AC'])
dfM['eHP'] = dfM.apply(lambda row: e5.effHP(row['adj HP'], row['eff AC'], 'linear'), axis=1)
dfM = dfM[['CR','eHP']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]

# plot adjusted damage per round
fig, y = plot_data_and_piecewise_fit(
    fig, dfM['CR'].to_numpy(), 
    dfM['eHP'].to_numpy(), 
    name='monsters', legendgroup='monsters', 
    line_color=COLOR_LIST[0],
    marker_color=COLOR_LIST[0],
    hovertemplate=
        '<b>monsters</b><br>'
        'CR %{x}<br>'+
        'eff HP %{y:,.1f}' + 
        '<extra></extra>'
)

# plot reference line
dfD0 = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','XP','HP Mean','AC','DPR Mean','AB'
dfD0['eHP'] = dfD0.apply(lambda row: e5.effDPR(row['HP Mean'], row['AC'], 'linear'), axis=1)
dfD0 = dfD0[dfD0['CR'].between(1,30)]
fig.add_trace(go.Scatter(
    x=dfD0['CR'], y=dfD0['eHP'],
    mode='markers+lines', 
    name='DMG',
    line=dict(color='black', dash='dash'),
    #hoverinfo='skip', 
    hovertemplate=
        '<b>DMG</b><br>'
        'CR %{x}<br>'+
        'eff DPR %{y:,.1f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    tfb.save_fig_html(fig, format='large', name='fig-eff-hp-vs-cr-large')
    tfb.save_fig_html(fig, format='small', name='fig-eff-hp-vs-cr-small')

/Users/tom/Development/python-venv/thefinishedbook/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning:

Covariance of the parameters could not be estimated



In [243]:
# create baseline table
import numpy as np
from scipy import optimize
import pandas as pd
from pprint import pprint
import sys
sys.path.append('../../assets/python/')
import estats5e as e5e


# get monster data
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM['unadj AC'] = dfM.apply(lambda row: unadjusted_armor_class(row), axis=1)
dfM['Ave Save'] = dfM.apply(lambda row: average_save_bonus(row), axis=1)
dfM['Ave Save'] = dfM['Ave Save'] + dfM.apply(lambda row: 4 if 'magic resistance' in row['AC adjustments'] else 0, axis=1)
dfM['eff AC'] = 0.5*(dfM['Ave Save']+14 + dfM['unadj AC'])

dfM = dfM[['CR','adj HP','AC','adj AC','adj DPR','adj AB','Ave Save','unadj AC']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]
dfM.sort_values(by='CR', inplace=True)



def fit_to_piecewise_linear(x, y, **kwargs):
    p, e = optimize.curve_fit(piecewise_linear, x, y)
    xp = kwargs.get('xp', np.linspace(1,30,30))
    return piecewise_linear(xp, *p)

def fit_to_linear(x, y):
    coefs = np.polyfit(x, y, 1)
    poly = np.poly1d(coefs)
    return poly(np.linspace(1,30,30))

xp = np.linspace(1,30,30).tolist()
xpm = [0.5] + [(xp[i] + xp[i+1])/2 for i in range(len(xp)-1)] + [30.5]
dpr = list(fit_to_piecewise_linear(dfM['CR'].to_numpy(), dfM['adj DPR'].to_numpy(), xp=np.array(xpm)))
dpr_range = [f'{np.ceil(dpr[i]):.0f} - {np.ceil(dpr[i+1])-1:.0f}' for i in range(len(xp))]

hp = list(fit_to_piecewise_linear(dfM['CR'].to_numpy(), dfM['adj HP'].to_numpy(), xp=np.array(xpm)))
hp_range = [f'{np.ceil(hp[i]):.0f} - {np.ceil(hp[i+1])-1:.0f}' for i in range(len(xp))]

n_round = 0
fit = {
    'CR':  list(range(1,31)),
    'Attack Bonus':  list(fit_to_linear(dfM['CR'].to_numpy(), dfM['adj AB'].to_numpy()).round(n_round)),
    'Armor Class':  list(fit_to_linear(dfM['CR'].to_numpy(), dfM['unadj AC'].to_numpy()).round(n_round)),
    'Save DC':  list(fit_to_linear(dfM['CR'].to_numpy(), dfM['adj AB'].to_numpy()+8).round(n_round)),
    'DPR Mean': list(fit_to_piecewise_linear(dfM['CR'].to_numpy(), dfM['adj DPR'].to_numpy())),
    'Damage / Round': dpr_range,
    'HP Mean':  list(fit_to_piecewise_linear(dfM['CR'].to_numpy(), dfM['adj HP'].to_numpy())),
    'Hit Points': hp_range,
    'Save Bonus':  list(fit_to_piecewise_linear(dfM['CR'].to_numpy(), dfM['Ave Save'].to_numpy()).round(n_round)),
    'XP':  [],
}
#fit['XP'] = list(e5e.effXP(
#    np.array(fit['HP Mean']), np.array(fit['Armor Class']), 
#    np.array(fit['DPR Mean']), np.array(fit['Attack Bonus'])).round(n_round))
fit['XP'] = [
    150,300,600,1000,1400,
    2000,2600,3500,4500,5700,
    6800,8200,10000,11500,13500,
    16000,18000,21000,23000,27000,
    34000,42000,52000,63000,73000,
    87000,102000,118000,133000,155000]

dfB = pd.DataFrame(fit)
dfB = dfB[['CR','Armor Class','Save Bonus','Hit Points','Attack Bonus','Save DC','Damage / Round','XP']]
dfB = dfB.style.format({
    'CR': '{:,.0f}',
    'Attack Bonus': '{:+,.0f}',
    'Armor Class': '{:,.0f}',
    'Save DC': '{:,.0f}',
    #'DPR Mean': '{:,.0f}',
    #'HP Mean': '{:,.0f}',
    'Save Bonus': '{:+,.0f}',
    'XP': '{:,.0f}',
}).hide().set_table_styles([
        {'selector': 'th', 'props': 'text-align: center;'},
        {'selector': 'td', 'props': 'text-align: center;;'},
    ], overwrite=False)
dfB.to_html(
    './tab-monster-baseline-stats.html',
    index=False, classes='wide', border=0)
dfB

/Users/tom/Development/python-venv/thefinishedbook/lib/python3.10/site-packages/scipy/optimize/_minpack_py.py:906: OptimizeWarning:

Covariance of the parameters could not be estimated



CR,Armor Class,Save Bonus,Hit Points,Attack Bonus,Save DC,Damage / Round,XP
1,14,+0,24 - 40,+5,13,12 - 17,150
2,14,+1,41 - 57,+5,13,18 - 22,300
3,15,+1,58 - 74,+5,13,23 - 28,600
4,15,+2,75 - 91,+6,14,29 - 34,"1,000"
5,15,+2,92 - 108,+6,14,35 - 40,"1,400"
6,16,+3,109 - 125,+7,15,41 - 46,"2,000"
7,16,+3,126 - 142,+7,15,47 - 51,"2,600"
8,16,+4,143 - 159,+8,16,52 - 57,"3,500"
9,17,+4,160 - 176,+8,16,58 - 63,"4,500"
10,17,+5,177 - 192,+9,17,64 - 69,"5,700"


In [ ]:
# plots probability of a monster hitting a PC with an attack
import json

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='hit probability',
            range=[0,1],
            tick0=0, dtick=0.2,
            minor=dict(tick0=0, dtick=0.1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

def attack_hit_crit_prob(AC, AB):
    return max(0.05, min(0.95, 0.05*(21 + AB - AC)))

# plot chance to hit vs monster targets
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 20)]
level = np.array(pc['level'])
ab = dfD['AB'].to_numpy()
ac = np.array(pc['armor class'])
prob = np.array([attack_hit_crit_prob(a, b) for a, b in zip(ac, ab)])
fig.add_trace(go.Scatter(
    x=level, 
    y=prob,
    mode='markers+lines', 
    name='monsters DMG',
    hovertemplate=
        'Level %{x}<br>'+
        'Hit Probability %{y:,.3f}' + 
        '<extra></extra>'
))

# plot chance to hit vs published monsters
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[['CR','AC','AB']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]
level = np.array(pc['level'])
ab = dfM['AB'].to_numpy()
ac = np.array(pc['armor class'])
prob = np.array([attack_hit_crit_prob(a, b) for a, b in zip(ac, ab)])
fig.add_trace(go.Scatter(
    x=level, 
    y=prob,
    mode='markers+lines', 
    name='monsters',
    hovertemplate=
        'Level %{x}<br>'+
        'Hit Probability %{y:,.3f}' + 
        '<extra></extra>'
))

# plot reference
fig.add_trace(go.Scatter(
    x=[1,20], 
    y=[0.65, 0.65],
    mode='lines', 
    name='monsters',
    showlegend=False,
    hoverinfo='skip',
    line=dict(color='black', dash='dash'),
))


# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-hit-prob-vs-level-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-hit-prob-vs-level-small')

In [206]:
# plots average save DC 
def save_fail_prob(SB, DC):
    return max(0.0, min(1.0, 0.05*(DC - SB - 1)))

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        margin=dict(l=60, r=25, b=55, t=20, pad=4),
        xaxis=dict(
            title_text='level',
            range=[0,21],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='save DC',
            range=[12,22],
            tick0=0, dtick=2,
            minor=dict(tick0=0, dtick=1),
        ),
        legend=dict(
            xanchor='left', yanchor='top',
            x=0.00, y=1.00,
            orientation='v',
            tracegroupgap=0,
        )
    )
)

# plot monster average
dfM = pd.read_csv('../../assets/data/monsters.csv')
dfM = dfM[['CR','AB','Save DC']].groupby(['CR']).mean().reset_index()
dfM = dfM[dfM['CR'].between(1, 30)]
plot_data_and_fit(
    fig, 
    #dfM['CR'], dfM['AB'] + 8, 
    dfM['CR'], dfM['Save DC'], 
    name='monsters (adj)',
    legendgroup='monsters (adj)',
    line=dict(color=COLOR_LIST[0]),
    hovertemplate=
        '<b>monsters (adj)</b><br>'+
        'CR %{x}<br>'+
        'Save DC %{y:,.0f}' + 
        '<extra></extra>'
)

# plot player character save DCs
fig.add_trace(go.Scatter(
    x=np.array(PC_DATA['level']), 
    y=np.array(PC_DATA['save dc']),
    mode='markers+lines', 
    name='player characters',
    line_color=COLOR_LIST[1],
    marker_color=COLOR_LIST[1],
    hovertemplate=
        '<b>player characters</p><br>'+
        'Level %{x}<br>'+
        'Save DC %{y:,.0f}' + 
        '<extra></extra>'
))

# plot monster save DCs
dfD = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','PB','AC','AB','DC'
dfD = dfD[dfD['CR'].between(1, 30)]
fig.add_trace(go.Scatter(
    x=np.array(dfD['CR']), 
    y=np.array(dfD['DC']),
    mode='markers+lines', 
    name='DMG',
    legendgroup='DMG',
    line=dict(color='black', dash='dash'),
    hovertemplate=
        '<b>DMG</p><br>'+
        'CR %{x}<br>'+
        'Save DC %{y:,.0f}' + 
        '<extra></extra>'
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-save-dc-vs-level-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-save-dc-vs-level-small')

In [9]:
# Plots eDPR for each class, calculated for a full adventuring day with Medium encounters and two short rests.
import pandas as pd
import sys
sys.path.append('../../assets/python/')
import estats5e as e5

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='PC rounds to win',
            range=[0, 5],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='v',
        )
    )
)

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}

# get PC data
classes = list(PC_CLASSES.keys())
encounter_difficulty = 'Medium'
pc_data_file = f'../../_classes/xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json'
df = load_pc_data(pc_data_file, ['class','level','effective damage per round mean'])
df = df[df['class'].isin(classes)]
dfG = df[['level','effective damage per round mean']].groupby(['level']).mean().reset_index()
levels = dfG['level'].to_numpy()
eDPRmean = dfG['effective damage per round mean'].to_numpy()


# plot DMG targets
dfD0 = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','XP','HP Mean','AC','DPR Mean','AB'
dfD0 = dfD0[dfD0['CR'].between(1,20)]
dfD0['eHP'] = dfD0.apply(lambda row: e5.effHP(row['HP Mean'], row['AC'], 'linear'), axis=1)

fig.add_trace(go.Scatter(
    x=dfD0['CR'], 
    y=dfD0['eHP']/(4*eDPRmean),
    mode='lines+markers', 
    #line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='DMG targets',
    hovertemplate=
            '<b>Level %{x}</b><br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>'
))

# plot published monsters
df0 = pd.read_csv('../../_monsters/calculating-monster-xp/monster-cr-and-xp.csv')
df0['eHP']  = df0.apply(lambda row: e5.effHP(row['adj HP'], row['adj AC'], 'linear'), axis=1)
df0['eDPR'] = df0.apply(lambda row: e5.effDPR(row['adj DPR'], row['adj AB'], 'linear'), axis=1)
df0 = df0[df0['CR'].between(1,20)]
#df0 = df0[df0['Book'].isin(['MM','MPMotM','FToD'])]
dfG = df0[['CR','eHP','eDPR']].groupby(['CR']).median().reset_index()

fig.add_trace(go.Scatter(
    x=dfG['CR'], 
    y=dfG['eHP']/(4*eDPRmean),
    mode='lines+markers', 
    #line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='published monsters',
    hovertemplate=
            '<b>Level %{x}</b><br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=[1,20], 
    y=[3,3],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-pcs-encounter-edpr-vs-monster-ehp-medium-adventuring-days-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-pcs-encounter-edpr-vs-monster-ehp-medium-adventuring-days-small')

In [210]:
# Plots eDPR for each class, calculated for a full adventuring day with Medium encounters and two short rests.
import pandas as pd
import sys
sys.path.append('../../assets/python/')
import estats5e as e5


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='NPC rounds to win',
            range=[0, 10],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='v',
        )
    )
)

# get PC data
PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}

classes = list(PC_CLASSES.keys())
encounter_difficulty = 'Medium'
pc_data_file = f'../../_classes/xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json'
df = load_pc_data(pc_data_file, ['class','level','encounter effective hit points mean'])
df = df[df['class'].isin(classes)]
dfG = df[['level','encounter effective hit points mean']].groupby(['level']).mean().reset_index()
levels = dfG['level'].to_numpy()
eHPmean = dfG['encounter effective hit points mean'].to_numpy()


# plot DMG targets
dfD0 = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','XP','HP Mean','AC','DPR Mean','AB'
dfD0 = dfD0[dfD0['CR'].between(1,20)]
dfD0['eDPR'] = dfD0.apply(lambda row: e5.effDPR(row['DPR Mean'], row['AB'], 'exp'), axis=1)

"""fig.add_trace(go.Scatter(
    x=dfD0['CR'], 
    y=(4*eHPmean)/dfD0['eDPR'],
    mode='lines+markers', 
    #line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='DMG targets',
    hovertemplate=
            '<b>Level %{x}</b><br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>'
))"""

# plot published monsters
df0 = pd.read_csv('../../_monsters/calculating-monster-xp/monster-cr-and-xp.csv')
df0['eHP']  = df0.apply(lambda row: e5.effHP(row['adj HP'], row['adj AC'], 'exp'), axis=1)
df0['eDPR'] = df0.apply(lambda row: e5.effDPR(row['adj DPR'], row['adj AB'], 'exp'), axis=1)
df0 = df0[df0['CR'].between(1,20)]
#df0 = df0[df0['Book'].isin(['MM','MPMotM','FToD'])]
dfG = df0[['CR','eHP','eDPR']].groupby(['CR']).median().reset_index()

fig.add_trace(go.Scatter(
    x=dfG['CR'], 
    y=(4*eHPmean)/dfG['eDPR'],
    mode='markers', 
    #line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='published monsters',
    hovertemplate=
            '<b>Level %{x}</b><br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>'
))

y_mean = np.mean((4*eHPmean)/dfG['eDPR'])

fig.add_trace(go.Scatter(
    x=[1,20], 
    y=[y_mean,y_mean],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-pcs-encounter-edpr-vs-monster-ehp-{encounter_difficulty.lower()}-adventuring-days-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-pcs-encounter-edpr-vs-monster-ehp-{encounter_difficulty.lower()}-adventuring-days-small')

In [217]:
# Plots eDPR for each class, calculated for a full adventuring day with Medium encounters and two short rests.
import pandas as pd
import sys
sys.path.append('../../assets/python/')
import estats5e as e5

# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='PC rounds to win',
            range=[0, 5],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='v',
        )
    )
)

PC_CLASSES = {
    'Barbarian': {'color': '#E7623E', 'group': 'martial'},
    'Bard': {'color': '#AB6DAC', 'group': 'spellcaster'},
    'Cleric': {'color': '#91A1B2', 'group': 'spellcaster'},
    'Druid': {'color': '#7A853B', 'group': 'spellcaster'},
    'Fighter': {'color': '#7F513E', 'group': 'martial'},
    'Monk': {'color': '#51A5C5', 'group': 'martial'},
    'Paladin': {'color': '#B59E54', 'group': 'martial-spellcaster'},
    'Ranger': {'color': '#507F62', 'group': 'martial-spellcaster'},
    'Rogue': {'color': '#555752', 'group': 'martial'},
    'Sorcerer': {'color': '#992E2E', 'group': 'spellcaster'},
    'Warlock': {'color': '#7B469B', 'group': 'spellcaster'},
    'Wizard': {'color': '#2A50A1', 'group': 'spellcaster'}
}

# get PC data
classes = list(PC_CLASSES.keys())
encounter_difficulty = 'Medium'
pc_data_file = f'../../_classes/xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json'
df = load_pc_data(pc_data_file, ['class','level','effective damage per round mean'])
df = df[df['class'].isin(classes)]
dfG = df[['level','effective damage per round mean']].groupby(['level']).median().reset_index()
levels = dfG['level'].to_numpy()
eDPRmean = dfG['effective damage per round mean'].to_numpy()


# plot DMG targets
dfD0 = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','XP','HP Mean','AC','DPR Mean','AB'
dfD0 = dfD0[dfD0['CR'].between(1,20)]
dfD0['eHP'] = dfD0.apply(lambda row: e5.effHP(row['HP Mean'], row['AC'], 'linear'), axis=1)

"""
fig.add_trace(go.Scatter(
    x=dfD0['CR'], 
    y=dfD0['HP Mean']/(4*0.65*eDPRmean),
    mode='lines+markers', 
    #line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='DMG targets',
    hovertemplate=
            '<b>Level %{x}</b><br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>'
))
"""

# plot published monsters
df0 = pd.read_csv('../../_monsters/calculating-monster-xp/monster-cr-and-xp.csv')
df0['eHP']  = df0.apply(lambda row: e5.effHP(row['adj HP'], row['adj AC'], 'linear'), axis=1)
df0['eDPR'] = df0.apply(lambda row: e5.effDPR(row['adj DPR'], row['adj AB'], 'linear'), axis=1)
df0 = df0[df0['CR'].between(1,20)]
#df0 = df0[df0['Book'].isin(['MM','MPMotM','FToD'])]
dfG = df0[['CR','eHP','eDPR','adj HP']].groupby(['CR']).median().reset_index()

fig.add_trace(go.Scatter(
    x=dfG['CR'], 
    #y=dfG['adj HP']/(4*0.65*eDPRmean),
    y=dfG['eHP']/(4*eDPRmean),
    mode='lines+markers', 
    #line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='published monsters',
    hovertemplate=
            '<b>Level %{x}</b><br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>'
))
y_mean = np.mean(dfG['eHP']/(4*eDPRmean))
print(y_mean)
fig.add_trace(go.Scatter(
    x=[1,20], 
    y=[y_mean,y_mean],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-pcs-encounter-edpr-vs-monster-ehp-medium-adventuring-days-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-pcs-encounter-edpr-vs-monster-ehp-medium-adventuring-days-small')

2.3456013886992606


In [12]:
# Plots eDPR for each class, calculated for a full adventuring day with Medium encounters and two short rests.
import pandas as pd
import sys
sys.path.append('../../assets/python/')
import estats5e as e5


# create figure
fig = go.Figure(
    layout=go.Layout(
        template=tfb.FIG_TEMPLATE,
        xaxis=dict(
            title_text='level',
            range=[0.8, 20.2],
            tick0=0, dtick=5,
            minor=dict(tick0=0, dtick=1),
        ),
        yaxis=dict(
            title_text='NPC rounds to win',
            range=[0, 10],
            tick0=0, dtick=1,
            minor=dict(tick0=0, dtick=0.5),
        ),
        legend=dict(
            yanchor='top',  y=0.99,
            xanchor='left', x=0.01,
            bgcolor='rgba(0,0,0,0)',
            orientation='v',
        )
    )
)

# get PC data
classes = list(PC_CLASSES.keys())
encounter_difficulty = 'Medium'
pc_data_file = f'../../_classes/xp-and-player-characters/pc data - {encounter_difficulty} 2 short rests.json'
df = load_pc_data(pc_data_file, ['class','level','hit points mean'])
df = df[df['class'].isin(classes)]
dfG = df[['level','hit points mean']].groupby(['level']).median().reset_index()
levels = dfG['level'].to_numpy()
eHPmean = dfG['hit points mean'].to_numpy()

# plot DMG targets
dfD0 = pd.read_csv('../../assets/data/dmg-targets.csv') # 'CR','XP','HP Mean','AC','DPR Mean','AB'
dfD0 = dfD0[dfD0['CR'].between(1,20)]
dfD0['eDPR'] = dfD0.apply(lambda row: e5.effDPR(row['DPR Mean'], row['AB'], 'exp'), axis=1)

fig.add_trace(go.Scatter(
    x=dfD0['CR'], 
    y=(4*eHPmean)/(0.65*dfD0['DPR Mean']),
    mode='lines+markers', 
    #line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='DMG targets',
    hovertemplate=
            '<b>Level %{x}</b><br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>'
))

# plot published monsters
df0 = pd.read_csv('../../_monsters/calculating-monster-xp/monster-cr-and-xp.csv')
df0['eHP']  = df0.apply(lambda row: e5.effHP(row['adj HP'], row['adj AC'], 'exp'), axis=1)
df0['eDPR'] = df0.apply(lambda row: e5.effDPR(row['adj DPR'], row['adj AB'], 'exp'), axis=1)
df0 = df0[df0['CR'].between(1,20)]
#df0 = df0[df0['Book'].isin(['MM','MPMotM','FToD'])]
dfG = df0[['CR','eHP','eDPR','adj DPR']].groupby(['CR']).median().reset_index()

fig.add_trace(go.Scatter(
    x=dfG['CR'], 
    y=(4*eHPmean)/(0.65*dfG['adj DPR']),
    mode='lines+markers', 
    #line=dict(color='black', dash='dash'),
    unselected=dict(marker=dict(opacity=0.0)),
    name='published monsters',
    hovertemplate=
            '<b>Level %{x}</b><br>'+
            'rounds %{y:.2f}'+
            '<extra></extra>'
))

fig.add_trace(go.Scatter(
    x=[1,20], 
    y=[3,3],
    mode='lines', 
    line=dict(color='black', dash='dash'),
    showlegend=False,
    hoverinfo='skip',
))

# show figure
fig.update_layout(width=600, height=450)
fig.show(config=tfb.FIG_CONFIG)

# save figures
if SAVEFIGS:
    fig.update_layout(autosize=True, width=None, height=None)
    #tfb.save_fig_html(fig, format='large', name=f'./fig-pcs-encounter-edpr-vs-monster-ehp-{encounter_difficulty.lower()}-adventuring-days-large')
    #tfb.save_fig_html(fig, format='small', name=f'./fig-pcs-encounter-edpr-vs-monster-ehp-{encounter_difficulty.lower()}-adventuring-days-small')